In [1]:
### Initial design of drawing a coplanar waveguide resonator 


# automatic reloading of modules when they change

%load_ext autoreload
%autoreload 2


In [2]:
# Import statements

import qiskit_metal as metal
from qiskit_metal import designs, draw
from qiskit_metal import MetalGUI, Dict, Headings
import pyEPR as epr

from qiskit_metal.qlibrary.terminations.launchpad_wb_coupled import LaunchpadWirebondCoupled
from qiskit_metal.qlibrary.terminations.open_to_ground import OpenToGround
from qiskit_metal.qlibrary.terminations.short_to_ground import ShortToGround
from qiskit_metal.qlibrary.tlines.meandered import RouteMeander
from qiskit_metal.qlibrary.tlines.straight_path import RouteStraight

In [3]:
# Instantiate QDesign class and initialise user interface

design = designs.DesignPlanar({}, True)
design.chips.main.size['size_x'] = '2mm'
design.chips.main.size['size_y'] = '2mm'
design.chips.main.size['size_z'] = '-500um'
design.chips.main.size['sample_holder_bottom'] = '500um'
design.chips.main.size['sample_holder_top'] = '2000um'


# Set hfss as renderer for design
hfss = design.renderers.hfss

In [4]:
# Allow overwriting
gui = MetalGUI(design)
design.overwrite_enabled = True
design.chips.main
#hfss.default_options

{'material': 'silicon',
 'layer_start': '0',
 'layer_end': '2048',
 'size': {'center_x': '0.0mm',
  'center_y': '0.0mm',
  'center_z': '0.0mm',
  'size_x': '2mm',
  'size_y': '2mm',
  'size_z': '-500um',
  'sample_holder_top': '2000um',
  'sample_holder_bottom': '500um'}}

In [5]:
# Create shorted pin and open pin for resonator

otg1 = OpenToGround(design, 'open_to_ground1', options=dict(pos_x='0um', pos_y='-1.25mm',
orientation='180'))
stg = ShortToGround(design, 'short_to_ground', options=dict(pos_x='0um', pos_y='0.5mm',
orientation='270')) 


# Jogs to allow bending of open end of CPW
# So far does not render in hfss
from collections import OrderedDict
jogs = OrderedDict()
jogs[0] = ["L", '125um']
jogs[1] = ["R", '100um']
#jogs[2] = ["L", '100um']

jogs2 = OrderedDict()
jogs2[0] = ["R", '100um']
jogs2[1] = ["L", '110um' ]

# Create meandered resonator between shorted and opened pin
rt_meander = RouteMeander(design, 'readout', Dict(
        total_length='8 mm',
        hfss_wire_bonds = False,
        fillet='70 um',
        lead = Dict(
        start_straight='0.1mm',
        end_straight='0mm'),
        #start_jogged_extensions=jogs2,
        #end_jogged_extension=jogs),
        meander=Dict(
        spacing = '150um'),
        pin_inputs=Dict(
            start_pin=Dict(component='short_to_ground', pin='short'),
            end_pin=Dict(component='open_to_ground1', pin='open')), ))

gui.rebuild()
gui.autoscale()


In [6]:
#Setup the launchpad location and orientation.
launch_options1 = dict(pos_x='-750um', pos_y='0.64mm', orientation='0')

lp = LaunchpadWirebondCoupled(design, 'P4_Q', options = launch_options1)

#stg = ShortToGround(design, 'short_to_ground',dict(pos_x='-500um', pos_y='1mm', orientation='0')) 


#Setup the OpenToGround location and orientation.
launch_options2 = dict(pos_x='0.75mm',  pos_y='0.64mm', orientation='180')

lp2 = LaunchpadWirebondCoupled(design, 'P5_Q', options = launch_options2)


#otg = OpenToGround(design, 'open_to_ground', options=dict(pos_x='0.75mm', pos_y='0.64mm',
#orientation='180'))

#After the two QComponents are added to design, connect them with a RouteMeander.
meander_options = Dict(
        total_length='1.5 mm',
        #fillet='200 um',
        #lead = dict(start_straight='100um', end_straight='100um'),
       # meander=Dict(
        #spacing = '600um'),
        pin_inputs=Dict(
            start_pin=Dict(component=lp.name, pin='tie'),
            end_pin=Dict(component=lp2.name, pin='tie')), )

meander = RouteStraight(design, 'feed',options=meander_options)

gui.rebuild()
gui.autoscale()

In [ ]:
# Screenshot the canvas only as a .png formatted file.
#gui.figure.savefig('shot.png')

#from IPython.display import Image, display
#_disp_ops = dict(width=500)
#display(Image('shot.png', **_disp_ops))

In [7]:
hfss.open_ansys(path = 'C:\Program Files\AnsysEM\AnsysEM19.3\Win64')

In [8]:
hfss.connect_ansys()

INFO 02:37PM [connect_project]: Connecting to Ansys Desktop API...
INFO 02:37PM [load_ansys_project]: 	Opened Ansys App
INFO 02:37PM [load_ansys_project]: 	Opened Ansys Desktop v2019.1.0
INFO 02:37PM [load_ansys_project]: 	Opened Ansys Project
	Folder:    C:/Users/2254729M/Documents/Ansoft/
	Project:   Project5
INFO 02:37PM [connect_design]: No active design found (or error getting active design).
INFO 02:37PM [connect]: 	 Connected to project "Project5". No design detected


In [9]:
hfss.activate_eigenmode_design("Readout")

02:37PM 48s WARNING [activate_eigenmode_design]: The name=Readout was not in active project.  A new design will be inserted to the project.  Names in active project are: 
[].  
INFO 02:37PM [connect_design]: 	Opened active design
	Design:    Readout [Solution type: Eigenmode]
WARNING 02:37PM [connect_setup]: 	No design setup detected.
WARNING 02:37PM [connect_setup]: 	Creating eigenmode default setup.
INFO 02:37PM [get_setup]: 	Opened setup `Setup`  (<class 'pyEPR.ansys.HfssEMSetup'>)


In [10]:
#hfss.render_design(['short_to_ground', 'readout', 'open_to_ground1', lp.name, lp2.name, 'feed'], [])
hfss.render_design([stg.name, otg1.name, 'readout'],[]) #, lp.name, lp2.name, 'feed'], [])
#hfss.save_screenshot()

In [ ]:
design._chips['main']['size']['size_y'] = '3mm'

In [ ]:
hfss.clean_active_design()
#hfss.render_design([], [], box_plus_buffer=False)

In [ ]:
hfss.subtract_from_ground()

In [11]:

setup = hfss.pinfo.setup
setup.passes = 20
print(f"""
Number of eigenmodes to find             = {setup.n_modes}
Number of simulation passes              = {setup.passes}
Convergence freq max delta percent diff  = {setup.delta_f}
""")

pinfo = hfss.pinfo
pinfo.design.set_variable('Lj', '10 nH')
pinfo.design.set_variable('Cj', '0 fF')
setup.analyze()

hfss.plot_convergences()

INFO 02:37PM [analyze]: Analyzing setup Setup



Number of eigenmodes to find             = 1
Number of simulation passes              = 20
Convergence freq max delta percent diff  = 0.1



02:41PM 28s INFO [hfss_report_f_convergence]: Saved convergences to C:\Users\2254729M\OneDrive - University of Glasgow\Documents\workspace\Thesis\Test Designs\hfss_eig_f_convergence.csv


In [ ]:
hfss.disconnect_ansys()